In [ ]:
# DS776 Auto-Update (runs in ~2 seconds, only updates when needed)
# If this cell fails, see Lessons/Course_Tools/AUTO_UPDATE_SYSTEM.md for help
%run ../../Lessons/Course_Tools/auto_update_introdl.py

# Homework 11: Text Generation and Decoding Strategies

**Total Points: 40**
- Part 1 (Decoding Strategies Comparison): 10 points
- Part 2 (Building API Helper Functions): 8 points
- Part 3 (Model Size Comparison): 8 points
- Part 4 (Creative Text Generation): 8 points
- Part 5 (Analysis and Comparison): 4 points
- Part 6 (Reflection): 2 points

## Part 1 - Decoding Strategies Comparison (10 points)

In this part, you'll experiment with different decoding strategies to see how they affect generated text quality and diversity.

**Tasks:**
1. Load a text generation model. **Suggested quantized models for A6000 GPUs (48GB VRAM):**

   **Small models (3-4B parameters, ~2-3GB VRAM):**
   - `"unsloth/Llama-3.2-3B-Instruct-bnb-4bit"` - Fast, good for testing
   - `"unsloth/Qwen2.5-3B-Instruct-bnb-4bit"` - Alternative 3B option

   **Medium models (7-8B parameters, ~5-6GB VRAM):**
   - `"unsloth/Llama-3.1-8B-Instruct-bnb-4bit"` - Balanced quality/speed
   - `"unsloth/Meta-Llama-3.1-8B-bnb-4bit"` - Base 8B variant
   - `"unsloth/Qwen2.5-7B-Instruct-bnb-4bit"` - Strong 7B model

   **Large models (14B parameters, ~8-10GB VRAM):**
   - `"unsloth/Qwen2.5-14B-Instruct-bnb-4bit"` - High quality generation

   **Very large models (70B+ parameters, ~40-45GB VRAM):**
   - `"unsloth/Meta-Llama-3.1-70B-bnb-4bit"` - Maximum quality (use if you have exclusive GPU access)

   **Note:** All these models use 4-bit quantization (bnb-4bit) which reduces memory by ~4x compared to full precision. Start with a 3B or 8B model for faster experimentation, then try larger models if you want to compare quality.

2. Create an interesting prompt for text generation (e.g., a story opening, a technical explanation, or a dialogue start)
3. Generate text using at least 5 different decoding configurations:
   - Greedy search (do_sample=False)
   - Beam search with num_beams=5
   - Beam search with no_repeat_ngram_size=2
   - Sampling with temperature=0.5
   - Sampling with temperature=1.5
   - Top-k sampling (top_k=50)
   - Nucleus sampling (top_p=0.9)

4. For each generated text:
   - Display the full generated text
   - Calculate and display the sequence log probability
   - Comment on the coherence, diversity, and quality

5. Write a summary (3-4 paragraphs) comparing the different decoding strategies. Which produced the most coherent text? Which was most creative? Which would you choose for different use cases?


## Part 1 - Decoding Strategies and Probability Analysis (10 points)

In this part, you'll implement three core decoding strategies and analyze how they affect both the generated text and the probability distributions. You'll learn to extract and visualize token probabilities from model outputs, giving you insight into how language models make decisions.

**Learning Objectives:**
- Understand how to extract probabilities from model logits
- Visualize probability distributions across decoding strategies
- Compare greedy search, beam search, and sampling methods
- Analyze the trade-offs between coherence, diversity, and model confidence


### Task 1.1: Understanding Model Outputs (2 points)

Before implementing decoding strategies, you need to understand how to extract and display token probabilities from the model.

**Instructions:**

1. Load a text generation model (use one of the quantized models suggested in the overview)
2. Define a prompt and get the model's output for the next token prediction
3. Complete the provided `get_top_tokens_with_probs()` function to:
   - Extract logits from model output
   - Convert logits to probabilities using softmax
   - Return the top-k tokens with their probabilities
4. Create a visualization showing the probability distribution
5. Answer the analysis questions below

**Hints:**
- Review Section 6 in the lesson for similar examples
- Use `torch.softmax()` to convert logits to probabilities
- Use `torch.topk()` to get the top tokens
- Probabilities should sum to approximately 1.0


In [ ]:
# YOUR CODE HERE
# Import necessary libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import matplotlib.pyplot as plt

# Load model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"  # or another model from suggestions

# YOUR CODE: Load tokenizer and model


# YOUR CODE: Define a prompt and tokenize it
prompt = "The future of artificial intelligence is"


def get_top_tokens_with_probs(model, input_ids, k=10):
    """
    Extract top-k tokens and their probabilities for next token prediction.

    Args:
        model: The language model
        input_ids: Tokenized input (tensor)
        k: Number of top tokens to return

    Returns:
        List of (token_string, probability) tuples

    Hints:
    - Get model output: output = model(input_ids)
    - Extract logits for last position: logits = output.logits[0, -1, :]
    - Convert to probabilities: probs = torch.softmax(logits, dim=-1)
    - Get top-k: top_probs, top_ids = torch.topk(probs, k=k)
    - Decode tokens: tokenizer.decode(token_id)
    """
    # YOUR CODE HERE
    pass


def plot_probability_distribution(probabilities, title="Token Probabilities"):
    """
    Plot probability distribution for top 50 tokens and cumulative probability.

    Hints:
    - Sort probabilities: sorted_probs, _ = torch.sort(probs, descending=True)
    - Use plt.bar() for histogram
    - Use torch.cumsum() for cumulative probability
    - Create a 2-subplot figure
    """
    # YOUR CODE HERE
    pass


# YOUR CODE: Call the functions and display results
# 1. Get top 10 tokens with probabilities
# 2. Print them in a formatted table
# 3. Create probability distribution plot


**Analysis Questions:**

1. **What is the probability of the most likely token?** Is it very confident (>50%) or more uncertain?

2. **How many tokens does it take to cover 90% of the probability mass?** This tells you how "focused" vs "spread out" the model's predictions are.

3. **Explain in your own words:** Why do we use `torch.softmax()` to convert logits to probabilities? What does softmax do?

📝 **YOUR ANSWERS HERE:**


### Task 1.2: Implement Three Decoding Strategies (5 points)

Now implement three core decoding strategies and compare their behavior. Use the **same prompt** for all three strategies so you can compare the results fairly.

**Strategy 1: Greedy Search**
- `do_sample=False` (no sampling parameter needed)
- Always picks the token with highest probability
- Deterministic - same output every time

**Strategy 2: Beam Search with No-Repeat N-Gram**
- `num_beams=5`
- `no_repeat_ngram_size=2` (prevents repetitive phrases)
- `do_sample=False`
- Explores multiple paths simultaneously

**Strategy 3: Nucleus (Top-P) Sampling**
- `do_sample=True`
- `top_p=0.9` (sample from tokens covering 90% probability mass)
- `temperature=0.7` (slightly lower than default for more focused generation)
- Stochastic - different output each time

**For each strategy:**
1. Generate text (max_length=100 or max_new_tokens=50)
2. Display the generated text
3. Show the top 5 token probabilities at **three key timesteps**:
   - Beginning (token 1 or 2 after prompt)
   - Middle (around token 25)
   - End (last few tokens)
4. Create a comparison visualization
5. Comment on the text quality and probability patterns


In [ ]:
# YOUR CODE HERE
# Strategy 1: Greedy Search

prompt = "In a shocking scientific discovery, researchers found"  # Use your own prompt
input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"].to(device)

print("=" * 70)
print("STRATEGY 1: GREEDY SEARCH")
print("=" * 70)

# Generate with greedy search
# YOUR CODE HERE

# Display generated text
# YOUR CODE HERE

# Show probability distributions at key timesteps
# Hint: You'll need to call the model again with the generated sequence
#       to get probabilities at different positions
# YOUR CODE HERE


In [ ]:
# YOUR CODE HERE
# Strategy 2: Beam Search with No-Repeat N-Gram

print("\n" + "=" * 70)
print("STRATEGY 2: BEAM SEARCH (num_beams=5, no_repeat_ngram_size=2)")
print("=" * 70)

# YOUR CODE HERE


In [ ]:
# YOUR CODE HERE
# Strategy 3: Nucleus (Top-P) Sampling

print("\n" + "=" * 70)
print("STRATEGY 3: NUCLEUS SAMPLING (top_p=0.9, temperature=0.7)")
print("=" * 70)

# YOUR CODE HERE


### Task 1.3: Comparison and Analysis (3 points)

Write 2-3 paragraphs comparing the three decoding strategies. Address:

**Text Quality:**
- Which strategy produced the most coherent text?
- Did any strategy produce repetitive text?
- Which was most creative/diverse?

**Probability Patterns:**
- How did the probability distributions differ between strategies?
- Did greedy search show higher confidence (more peaked distributions)?
- How did sampling affect the probability spread?
- Did you notice the distributions changing from beginning to end of generation?

**Use Case Recommendations:**
- When would you use greedy search?
- When would beam search be preferred?
- When would you choose nucleus sampling?
- What trade-offs exist between coherence and diversity?

📝 **YOUR ANALYSIS HERE:**


## Part 2 - Extending the Conversation Class with Decoding Parameters (8 points)

In Lesson 11, Section 5.5, you learned about the `Conversation` class for building chatbots with APIs. In Section 7.5, you learned about decoding parameters (temperature, top_p, max_tokens) that control text generation behavior.

In this part, you'll combine these concepts by extending the `Conversation` class to support decoding parameter control, allowing you to fine-tune the chatbot's behavior for different use cases.

**Learning Objectives:**
- Practice working with chat roles (system/user/assistant) and conversation history
- Understand how decoding parameters affect model behavior
- Apply object-oriented design patterns to build maintainable APIs
- Test different parameter combinations for various tasks


### Task 2.1: Copy and Extend the Conversation Class (3 points)

**Instructions:**

1. Copy the `Conversation` class from Lesson 11, Section 5.5 (you'll find it in the "Building a Chatbot Class" section)

2. Extend the class to support decoding parameters:
   - Add `temperature`, `top_p`, and `max_tokens` parameters to `__init__()` with these defaults:
     - `temperature=0.7`
     - `top_p=0.9`
     - `max_tokens=256`
   - Modify the `send()` method to pass these parameters to the API call
   - Allow per-message parameter overrides in `send()` (e.g., `send(message, temperature=0.0)`)

3. Your extended class should:
   - Store default parameters as instance variables
   - Use defaults if no overrides provided
   - Pass all three parameters to `client.chat.completions.create()`

**Hints:**
- The original `Conversation` class already handles the OpenAI client and message history
- You just need to add parameter handling to `__init__()` and `send()`
- Use `if parameter is not None` to check for overrides


In [ ]:
# YOUR CODE HERE
# Import necessary libraries
import os
from openai import OpenAI

# Copy and extend the Conversation class
class Conversation:
    """
    Extended chatbot class with decoding parameter control.

    Args:
        api_key (str): OpenRouter API key
        model (str): Model name (e.g., 'google/gemini-2.5-flash-lite')
        system_prompt (str, optional): Initial instructions for the model
        temperature (float): Sampling temperature (default: 0.7)
        top_p (float): Nucleus sampling parameter (default: 0.9)
        max_tokens (int): Maximum tokens to generate (default: 256)
    """

    def __init__(self, api_key, model="google/gemini-2.5-flash-lite",
                 system_prompt=None, temperature=0.7, top_p=0.9, max_tokens=256):
        # YOUR CODE HERE
        pass

    def send(self, user_message, temperature=None, top_p=None, max_tokens=None):
        """
        Send a message and get a response.

        Args:
            user_message (str): The user's input
            temperature (float, optional): Override default temperature
            top_p (float, optional): Override default top_p
            max_tokens (int, optional): Override default max_tokens

        Returns:
            str: The assistant's response
        """
        # YOUR CODE HERE
        pass

    def display(self, show_recent_only=True):
        """Show conversation history (you can use a simple print for now)"""
        # YOUR CODE HERE
        pass

    def reset(self, system_prompt=None):
        """Reset conversation history"""
        # YOUR CODE HERE
        pass

print("✓ Extended Conversation class defined")


### Task 2.2: Test Different Parameter Recipes (3 points)

**Instructions:**

Create three different chatbot configurations optimized for different use cases:

**Recipe 1: Factual Question Answering**
- `temperature=0` (deterministic/greedy)
- `top_p=1.0` (not used when temperature=0)
- `max_tokens=150`
- System prompt: "You are a knowledgeable assistant providing accurate, factual information."
- Test with a factual question (e.g., "What is photosynthesis?")

**Recipe 2: Creative Writing**
- `temperature=1.2` (more random)
- `top_p=0.95` (wide vocabulary)
- `max_tokens=300`
- System prompt: "You are a creative storyteller with a vivid imagination."
- Test with a creative prompt (e.g., "Start a science fiction story about AI")

**Recipe 3: General Conversation**
- `temperature=0.7` (balanced)
- `top_p=0.9` (nucleus sampling)
- `max_tokens=200`
- System prompt: "You are a friendly, helpful assistant."
- Test with a conversational prompt (e.g., "Explain neural networks to a beginner")

For each recipe:
1. Create a new `Conversation` instance with the specified parameters
2. Send at least 2 messages to create a multi-turn conversation
3. Display the conversation (or print the messages)
4. Note the characteristics of the responses (consistency, creativity, length)


In [ ]:
# YOUR CODE HERE
# Recipe 1: Factual Question Answering

print("=" * 70)
print("RECIPE 1: FACTUAL QUESTION ANSWERING")
print("(temperature=0, top_p=1.0, max_tokens=150)")
print("=" * 70)

# Create chatbot
factual_chat = Conversation(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    model="google/gemini-2.5-flash-lite",
    system_prompt="You are a knowledgeable assistant providing accurate, factual information.",
    temperature=0,
    top_p=1.0,
    max_tokens=150
)

# Test with multi-turn conversation
# YOUR CODE HERE - send at least 2 messages and display responses


In [ ]:
# YOUR CODE HERE
# Recipe 2: Creative Writing

print("\n" + "=" * 70)
print("RECIPE 2: CREATIVE WRITING")
print("(temperature=1.2, top_p=0.95, max_tokens=300)")
print("=" * 70)

# YOUR CODE HERE - create chatbot and test with creative prompts


In [ ]:
# YOUR CODE HERE
# Recipe 3: General Conversation

print("\n" + "=" * 70)
print("RECIPE 3: GENERAL CONVERSATION")
print("(temperature=0.7, top_p=0.9, max_tokens=200)")
print("=" * 70)

# YOUR CODE HERE - create chatbot and test with conversational prompts


### Task 2.3: Analysis and Comparison (2 points)

**Write 2-3 paragraphs addressing:**

1. **Parameter Effects:**
   - How did temperature affect the responses? Compare the factual (temp=0) vs creative (temp=1.2) outputs.
   - What role did `top_p` play? Was the difference noticeable?
   - How did `max_tokens` affect response completeness?

2. **Use Case Recommendations:**
   - Which recipe worked best for each type of task?
   - When would you choose low temperature vs high temperature?
   - What trade-offs did you observe between consistency and creativity?

3. **Conversation History:**
   - Did the chatbot maintain context across multiple turns?
   - How does sending the full conversation history enable this?
   - What are the cost implications of longer conversations?


📝 **YOUR ANALYSIS HERE:**


## Part 3 - Model Size Comparison (8 points)

Different model sizes offer different trade-offs between quality and computational cost. In this part, you'll compare text generation across different model sizes.

**Tasks:**
1. Load at least two different-sized models (e.g., 3B and 8B parameter models). Suggested options:
   - "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" (3B parameters, 4-bit quantized)
   - "unsloth/Llama-3.1-8B-Instruct-bnb-4bit" (8B parameters, 4-bit quantized)
   
2. Create 3 diverse prompts that test different capabilities:
   - A factual knowledge question
   - A creative writing task
   - A reasoning/problem-solving task
   
3. Generate responses from each model for each prompt using the same decoding parameters (e.g., temperature=0.7, top_p=0.9)

4. For each generation, measure and record:
   - Generation time
   - Memory usage (if possible)
   - Response length
   - Qualitative assessment of quality
   
5. Create visualizations:
   - Bar chart comparing generation times
   - Any other relevant comparisons
   
6. Write an analysis (2-3 paragraphs) discussing:
   - Performance differences between model sizes
   - Quality differences in outputs
   - When you would choose each model size
   - Trade-offs between speed and quality

In [ ]:
# YOUR CODE HERE
# Load different-sized models

In [ ]:
# YOUR CODE HERE
# Create prompts and generate responses with timing

In [ ]:
# YOUR CODE HERE
# Create visualizations comparing performance

📝 **YOUR ANALYSIS HERE:**

## Part 4 - Creative Text Generation Application (8 points)

Apply what you've learned about text generation to build a creative application. Choose ONE of the following:

### Option A: Story Continuation System
- Create a system that takes a story opening and generates multiple continuations using different decoding strategies
- Allow the user to select which continuation they prefer
- Continue the story iteratively, building on the selected continuations
- Generate at least 3 rounds of story development

### Option B: Dialogue Generator
- Create a multi-character dialogue system
- Define personalities for 2-3 characters (via system prompts)
- Generate a conversation between the characters on a given topic
- Experiment with different decoding parameters for each character to reflect their personality

### Option C: Writing Style Transformer
- Take a piece of text and rewrite it in different styles (formal, casual, poetic, technical, etc.)
- Use both local models and API-based models
- Compare how different models handle style transformation
- Test with at least 3 different input texts and 4 different target styles

**Requirements:**
- Use at least 2 different models (one local, one via API)
- Experiment with at least 3 different decoding configurations
- Include clear output formatting and labeling
- Write a reflection (2-3 paragraphs) on what worked well and what didn't, and what you learned about text generation from this exercise

In [ ]:
# YOUR CODE HERE
# Implement your chosen creative application

📝 **YOUR REFLECTION HERE:**

## Part 5 - Analysis and Comparison (4 points)

Synthesize your findings from all the previous parts into a comprehensive analysis.

**Write 2-3 paragraphs addressing:**

1. **Local vs. API-Based Generation:**
   - Compare the experience of using local models vs. API-based models
   - Discuss trade-offs in terms of control, cost, latency, and quality
   - When would you choose each approach for a production application?

2. **Decoding Strategy Selection:**
   - Which decoding strategies performed best for different types of tasks?
   - How did you balance coherence and diversity in your experiments?
   - What general principles would you follow when choosing decoding parameters?

3. **Practical Applications:**
   - What real-world applications would benefit from the techniques you explored?
   - What challenges did you encounter that would need to be addressed for production use?
   - What additional features or improvements would make your implementations more robust?

📝 **YOUR ANALYSIS HERE:**

## Part 6 - Reflection (2 points)

1. What, if anything, did you find difficult to understand for the lesson? Why?

📝 **YOUR ANSWER HERE:**

2. What resources did you find supported your learning most and least for this lesson? (Be honest - I use your input to shape the course.)

📝 **YOUR ANSWER HERE:**

### Export Notebook to HTML for Canvas Upload

Uncomment the two lines below and run the cell to export the current notebook to HTML.

In [ ]:
# from introdl import export_this_to_html
# export_this_to_html()